Handling imbalanced data in customer churn prediction

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/14_imbalanced/customer_churn.csv")
df.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4673,6402-ZFPPI,Female,1,No,No,25,Yes,Yes,Fiber optic,Yes,...,No,No,Yes,Yes,Month-to-month,Yes,Mailed check,102.80,2660.2,Yes
2000,7020-OZKXZ,Female,1,No,No,9,Yes,No,Fiber optic,No,...,Yes,No,No,No,Month-to-month,No,Credit card (automatic),75.50,637.4,No
5144,1078-TDCRN,Female,1,Yes,No,3,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,30.75,82.85,No
915,6513-EECDB,Male,1,Yes,No,1,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,73.55,73.55,Yes
5330,5747-PMBSQ,Male,1,Yes,No,10,Yes,No,Fiber optic,No,...,Yes,Yes,Yes,No,Month-to-month,Yes,Mailed check,92.50,934.1,Yes


In [3]:

df.drop('customerID',axis='columns',inplace=True)

In [4]:
df1 = df[df.TotalCharges!=' ']
df1.shape

(7032, 20)

In [5]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_8924\973151263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.TotalCharges = pd.to_numeric(df1.TotalCharges)


In [6]:
df1.replace('No internet service','No',inplace=True)
df1.replace('No phone service','No',inplace=True)

C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_8924\2045096646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.replace('No internet service','No',inplace=True)
C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_8924\2045096646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.replace('No phone service','No',inplace=True)


In [7]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']
for col in yes_no_columns:
    df1[col].replace({'Yes': 1,'No': 0},inplace=True)

C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_8924\1648037665.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col].replace({'Yes': 1,'No': 0},inplace=True)


In [8]:
df1['gender'].replace({'Female':1,'Male':0},inplace=True)

C:\Users\Anjana Kavidu\AppData\Local\Temp\ipykernel_8924\698335744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['gender'].replace({'Female':1,'Male':0},inplace=True)


In [9]:
df2 = pd.get_dummies(data=df1, columns=['InternetService','Contract','PaymentMethod'])
df2.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype='object')

In [10]:
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df2[cols_to_scale] = scaler.fit_transform(df2[cols_to_scale])

In [11]:
X = df2.drop('Churn',axis='columns')
y = testLabels = df2.Churn.astype(np.float32)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)
y_train.value_counts()

0.0    4130
1.0    1495
Name: Churn, dtype: int64

Build an ANN model

In [14]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [29]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
    model = keras.Sequential([
        keras.layers.Dense(26, input_dim=26, activation='relu'),
        keras.layers.Dense(15, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    if weights == -1:
        model.fit(X_train, y_train, epochs=100)
    else:
        model.fit(X_train, y_train, epochs=100, class_weight = weights)
    
    print(model.evaluate(X_test, y_test))
    
    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)
    
    
    return y_preds

In [19]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
176/176 [==============================] - 1s 3ms/step - loss: 0.5027 - accuracy: 0.7460
Epoch 2/100
176/176 [==============================] - 0s 3ms/step - loss: 0.4275 - accuracy: 0.7931
Epoch 3/100
176/176 [==============================] - 0s 3ms/step - loss: 0.4203 - accuracy: 0.7982
Epoch 4/100
176/176 [==============================] - 1s 3ms/step - loss: 0.4163 - accuracy: 0.7984
Epoch 5/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4138 - accuracy: 0.8000
Epoch 6/100
176/176 [==============================] - 1s 3ms/step - loss: 0.4111 - accuracy: 0.8018
Epoch 7/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4098 - accuracy: 0.8052
Epoch 8/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4094 - accuracy: 0.8021
Epoch 9/100
176/176 [==============================] - 0s 3ms/step - loss: 0.4070 - accuracy: 0.8011
Epoch 10/100
176/176 [==============================] - 0s 2ms/step - loss: 0.4053 - accura

In [20]:
print("Classification Report: \n", classification_report(y_test, y_preds))

Classification Report: 
               precision    recall  f1-score   support

         0.0       0.82      0.91      0.86      1033
         1.0       0.63      0.44      0.52       374

    accuracy                           0.78      1407
   macro avg       0.72      0.67      0.69      1407
weighted avg       0.77      0.78      0.77      1407



# __Lets handle imbalanced data__

__Method 1: Undersampling__

In [21]:
count_class_0, count_class_1 = df1.Churn.value_counts()

df_class_0 = df2[df2['Churn'] == 0]
df_class_1 = df2[df2['Churn'] == 1]

In [24]:
count_class_0, count_class_1

(5163, 1869)

In [25]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under sampling')
print(df_test_under.Churn.value_counts())

Random under sampling
0    1869
1    1869
Name: Churn, dtype: int64


In [27]:
X = df_test_under.drop('Churn', axis=1)
y = df_test_under['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [28]:
y_train.value_counts()

1    1495
0    1495
Name: Churn, dtype: int64

In [30]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 1s 2ms/step - loss: 0.6522 - accuracy: 0.6365
Epoch 2/100
94/94 [==============================] - 0s 2ms/step - loss: 0.5243 - accuracy: 0.7552
Epoch 3/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4982 - accuracy: 0.7632
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4910 - accuracy: 0.7689
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4856 - accuracy: 0.7682
Epoch 6/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4828 - accuracy: 0.7699
Epoch 7/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4806 - accuracy: 0.7672
Epoch 8/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4775 - accuracy: 0.7706
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4767 - accuracy: 0.7726
Epoch 10/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4745 - accuracy: 0.7726
Epoch 11/

In [31]:
print("Classification Report: \n", classification_report(y_test, y_preds))

Classification Report: 
               precision    recall  f1-score   support

           0       0.79      0.68      0.73       374
           1       0.72      0.82      0.77       374

    accuracy                           0.75       748
   macro avg       0.76      0.75      0.75       748
weighted avg       0.76      0.75      0.75       748



__Method 2 : Oversampling__

In [34]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_1_over, df_class_0], axis=0)

print('Random over sampling')
print(df_test_over.Churn.value_counts())

Random over sampling
1    5163
0    5163
Name: Churn, dtype: int64


In [37]:
X = df_test_over.drop('Churn', axis=1)
y = df_test_over['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [38]:
y_train.value_counts()

1    4130
0    4130
Name: Churn, dtype: int64

In [39]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 2ms/step - loss: 0.5392 - accuracy: 0.7372
Epoch 2/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4806 - accuracy: 0.7671
Epoch 3/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4745 - accuracy: 0.7684
Epoch 4/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4696 - accuracy: 0.7702
Epoch 5/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4665 - accuracy: 0.7759
Epoch 6/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4646 - accuracy: 0.7749
Epoch 7/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4608 - accuracy: 0.7792
Epoch 8/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4578 - accuracy: 0.7805
Epoch 9/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4559 - accuracy: 0.7844
Epoch 10/100
259/259 [==============================] - 0s 1ms/step - loss: 0.4539 - accura

In [40]:
print("Classification Report: \n", classification_report(y_test, y_preds))

Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.73      0.78      1033
           1       0.76      0.85      0.80      1033

    accuracy                           0.79      2066
   macro avg       0.80      0.79      0.79      2066
weighted avg       0.80      0.79      0.79      2066



__Method 3: SMOTE__

In [41]:
X = df2.drop('Churn', axis=1)
y = df2['Churn']

In [45]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

0    5163
1    5163
Name: Churn, dtype: int64

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, stratify=y_sm)

In [49]:
y_train.value_counts()

0    4130
1    4130
Name: Churn, dtype: int64

In [50]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
259/259 [==============================] - 1s 3ms/step - loss: 0.5289 - accuracy: 0.7476
Epoch 2/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4746 - accuracy: 0.7726
Epoch 3/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4657 - accuracy: 0.7764
Epoch 4/100
259/259 [==============================] - 1s 4ms/step - loss: 0.4597 - accuracy: 0.7814
Epoch 5/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4547 - accuracy: 0.7851
Epoch 6/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4478 - accuracy: 0.7875
Epoch 7/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4441 - accuracy: 0.7896
Epoch 8/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4394 - accuracy: 0.7936
Epoch 9/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4339 - accuracy: 0.7971
Epoch 10/100
259/259 [==============================] - 1s 2ms/step - loss: 0.4297 - accura

In [51]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.84      0.78      0.81      1033
           1       0.79      0.85      0.82      1033

    accuracy                           0.81      2066
   macro avg       0.82      0.81      0.81      2066
weighted avg       0.82      0.81      0.81      2066



__Method 4: Use of ensamble with undersampling__

In [52]:
df2.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [53]:
X = df2.drop('Churn', axis=1)
y = df2['Churn']

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [57]:
y_train.value_counts()

0    4130
1    1495
Name: Churn, dtype: int64

In [58]:
df3 = X_train.copy()
df3['Churn'] = y_train

In [59]:
df3.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
755,0,1,0,0,0.267606,1,1,0,0,1,...,1,0,1,0,0,0,0,1,0,1
4457,0,0,1,1,0.295775,1,0,1,0,0,...,1,0,1,0,0,1,0,0,0,0
5744,0,1,1,0,0.549296,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4006,0,0,1,1,1.000000,1,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
5848,0,0,1,1,1.000000,1,1,1,1,1,...,1,0,0,0,1,0,1,0,0,0


In [60]:
df3_class_0 = df3[df3['Churn'] == 0]
df3_class_1 = df3[df3['Churn'] == 1]

In [62]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Churn', axis=1)
    y_train = df_train.Churn

    return X_train, y_train

In [67]:
X_train, y_train = get_train_batch(df3_class_0, df3_class_1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 1s 2ms/step - loss: 0.6106 - accuracy: 0.6699
Epoch 2/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5030 - accuracy: 0.7642
Epoch 3/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4885 - accuracy: 0.7736
Epoch 4/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4837 - accuracy: 0.7722
Epoch 5/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4796 - accuracy: 0.7692
Epoch 6/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4769 - accuracy: 0.7686
Epoch 7/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4747 - accuracy: 0.7699
Epoch 8/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4729 - accuracy: 0.7726
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4700 - accuracy: 0.7746
Epoch 10/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4685 - accuracy: 0.7746
Epoch 11/

In [68]:
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

           0       0.91      0.71      0.80      1033
           1       0.50      0.81      0.62       374

    accuracy                           0.74      1407
   macro avg       0.71      0.76      0.71      1407
weighted avg       0.80      0.74      0.75      1407



In [69]:
X_train, y_train = get_train_batch(df3_class_0, df3_class_1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
94/94 [==============================] - 1s 2ms/step - loss: 0.6251 - accuracy: 0.6836
Epoch 2/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5443 - accuracy: 0.7361
Epoch 3/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5185 - accuracy: 0.7502
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5073 - accuracy: 0.7515
Epoch 5/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5023 - accuracy: 0.7575
Epoch 6/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4989 - accuracy: 0.7562
Epoch 7/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4968 - accuracy: 0.7569
Epoch 8/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4956 - accuracy: 0.7589
Epoch 9/100
94/94 [==============================] - 0s 2ms/step - loss: 0.4919 - accuracy: 0.7585
Epoch 10/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4902 - accuracy: 0.7612
Epoch 11/

In [70]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.89      0.71      0.79      1033
           1       0.48      0.75      0.58       374

    accuracy                           0.72      1407
   macro avg       0.68      0.73      0.69      1407
weighted avg       0.78      0.72      0.73      1407



In [72]:
X_train, y_train = get_train_batch(df3_class_0, df3_class_1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
83/83 [==============================] - 1s 2ms/step - loss: 0.6108 - accuracy: 0.6725
Epoch 2/100
83/83 [==============================] - 0s 3ms/step - loss: 0.5218 - accuracy: 0.7651
Epoch 3/100
83/83 [==============================] - 0s 2ms/step - loss: 0.5001 - accuracy: 0.7685
Epoch 4/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4906 - accuracy: 0.7727
Epoch 5/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4837 - accuracy: 0.7742
Epoch 6/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4784 - accuracy: 0.7772
Epoch 7/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4750 - accuracy: 0.7742
Epoch 8/100
83/83 [==============================] - 0s 4ms/step - loss: 0.4709 - accuracy: 0.7784
Epoch 9/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4678 - accuracy: 0.7799
Epoch 10/100
83/83 [==============================] - 0s 2ms/step - loss: 0.4658 - accuracy: 0.7822
Epoch 11/

In [73]:
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.91      0.66      0.76      1033
           1       0.46      0.82      0.59       374

    accuracy                           0.70      1407
   macro avg       0.69      0.74      0.68      1407
weighted avg       0.79      0.70      0.72      1407



In [78]:
len(y_pred1)

1407

In [79]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [80]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.92      0.69      0.79      1033
           1       0.49      0.83      0.62       374

    accuracy                           0.73      1407
   macro avg       0.71      0.76      0.70      1407
weighted avg       0.81      0.73      0.74      1407

